In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
import keras.backend as K
from typing import Optional

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,

        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [4]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI_AD/train',
                                                   target_size = (224,224),
                                                   class_mode = 'binary',
                                                   subset = 'training',
                                                   batch_size = 32)

Found 352 images belonging to 2 classes.


In [5]:
test_dataset = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/Colab Notebooks/MCI_AD/test',
                                                  target_size = (224,224),
                                                  class_mode = 'binary',
                                                  batch_size = 32)

Found 130 images belonging to 2 classes.


In [6]:
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


In [7]:
for layer in base_model.layers:
    layer.trainable=False


In [8]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dense(1, activation='sigmoid'))

In [9]:
def f1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [10]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) 

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [11]:

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]


In [12]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=METRICS)

In [13]:
history=model.fit(train_dataset,
                        epochs = 30,
                        verbose = 1,
                         callbacks=lr_scheduler)

Epoch 1/30
11/11 [==============================] - 23s 515ms/step - loss: 0.7038 - accuracy: 0.6477 - precision: 0.6625 - recall: 0.6023 - auc: 0.6820 - f1_score: 0.6161 - lr: 0.0100
Epoch 2/30
11/11 [==============================] - 6s 514ms/step - loss: 0.5028 - accuracy: 0.7642 - precision: 0.7409 - recall: 0.8125 - auc: 0.8360 - f1_score: 0.7750 - lr: 0.0063
Epoch 3/30
11/11 [==============================] - 7s 602ms/step - loss: 0.4817 - accuracy: 0.7926 - precision: 0.7814 - recall: 0.8125 - auc: 0.8515 - f1_score: 0.7965 - lr: 0.0040
Epoch 4/30
11/11 [==============================] - 6s 549ms/step - loss: 0.4384 - accuracy: 0.8153 - precision: 0.8000 - recall: 0.8409 - auc: 0.8739 - f1_score: 0.8172 - lr: 0.0025
Epoch 5/30
11/11 [==============================] - 6s 514ms/step - loss: 0.4605 - accuracy: 0.7983 - precision: 0.7838 - recall: 0.8239 - auc: 0.8616 - f1_score: 0.7937 - lr: 0.0016
Epoch 6/30
11/11 [==============================] - 7s 671ms/step - loss: 0.4029 - a

In [14]:
scores = model.evaluate(test_dataset)

5/5 [==============================] - 5s 414ms/step - loss: 0.2541 - accuracy: 0.9154 - precision: 0.8764 - recall: 1.0000 - auc: 0.9459 - f1_score: 0.9461


In [15]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

Accuracy =  0.9153845906257629
Precision =  0.8764045238494873
Recall =  1.0
AUC =  0.9458826780319214
F1_score =  0.9460954666137695
